In [ ]:
import pandas as pd
import joblib
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss

In [ ]:
def predict_top_n_tags(text, vectorizer, classifier, multilabel_binarizer, n=5):
    """
    Predice las n etiquetas más probables para un texto dado.

    Parámetros:
    -----------
    text : str
        Texto para el cual predecir etiquetas.
    vectorizer : TfidfVectorizer
        Vectorizador TF-IDF entrenado.
    classifier : OneVsRestClassifier
        Clasificador entrenado.
    multilabel_binarizer : MultiLabelBinarizer
        Binarizador de etiquetas utilizado.
    n : int
        Número máximo de etiquetas a predecir.

    Retorna:
    --------
    list
        Lista de las n etiquetas más probables.
    """
    X = vectorizer.transform([text.lower()])

    y_proba = classifier.predict_proba(X)

    top_n_indices = y_proba[0].argsort()[-n:][::-1]

    y_pred = np.zeros(y_proba.shape[1], dtype=int)
    for idx in top_n_indices:
        if y_proba[0][idx] > 0.05:
            y_pred[idx] = 1

    predicted_tags = multilabel_binarizer.inverse_transform(y_pred.reshape(1, -1))

    return predicted_tags[0]

In [ ]:
clf = joblib.load('../models/modelo_logreg.joblib')
vectorizer = joblib.load('../models/vectorizer_tfidf.joblib')
multilabel_binarizer = joblib.load('../models/multilabel_binarizer.joblib')

In [ ]:
data_muestra = pd.read_parquet('../data/output/muestras_100_preguntas.parquet')

In [ ]:
data_muestra['text'] = data_muestra['Title'] + ' ' + data_muestra['Body'].fillna('')
X_muestra = vectorizer.transform(data_muestra['text'].astype(str).str.lower())

In [ ]:
resultados = []
for idx, row in data_muestra.iterrows():
    texto = row['text']

    tags_pred = predict_top_n_tags(texto, vectorizer, clf, multilabel_binarizer, n=5)

    resultados.append({
        'ID': row['Id'],
        'Título': row['Title'],
        'Etiquetas_reales': row['Tag'],
        'Etiquetas_predichas': tags_pred,
        'Acierto': any(tag in row['Tag'] for tag in tags_pred) if len(tags_pred) > 0 else False
    })

In [ ]:
df_resultados = pd.DataFrame(resultados)
print("\n--- Resumen de predicciones ---")
print(f"Total de muestras: {len(df_resultados)}")
print(f"Predicciones con al menos un acierto: {df_resultados['Acierto'].sum()}")
print(f"Precisión global: {df_resultados['Acierto'].mean():.2f}")

In [ ]:
print("\n--- Ejemplos de predicciones ---")
for i in range(min(5, len(df_resultados))):
    print(f"\nEjemplo {i+1}:")
    print(f"Título: {df_resultados.iloc[i]['Título']}")
    print(f"Etiquetas reales: {df_resultados.iloc[i]['Etiquetas_reales']}")
    print(f"Etiquetas predichas: {df_resultados.iloc[i]['Etiquetas_predichas']}")